In [ ]:
import xarray as xr
import fsspec
import zarr
import yaml
import scipy.constants
import numpy as np

import holoviews as hv
import geoviews as gv
import geoviews.feature as gf
from cartopy import crs
import hvplot.xarray

hv.extension('bokeh')
epsg_3031 = crs.Stereographic(central_latitude=-90, true_scale_latitude=-71)
latlon = crs.PlateCarree()

### Load parameters

In [ ]:
with open("data_preprocessing_config.yaml", "r") as f:
    config = yaml.safe_load(f)

for k in config:
    print(f"=== {k} ===")
    for sk in config[k]:
        print(f"- {sk}: {config[k][sk]}")
    print("")

### Find all processed data cache files

In [ ]:
output_storage_location = config['output']['processed_flight_cache_url']

# Open the directory with fsspec and list all *.zarr.zip files
fs, path = fsspec.core.url_to_fs(output_storage_location)
zarr_files = fs.glob(f"{path}/*.zarr")

print(f"Found {len(zarr_files)} zarr files in {output_storage_location} ({type(fs).__name__})")

In [ ]:
datasets = []
broken_datasets = []
for path in zarr_files:
    mapper = fsspec.get_mapper(f"{path}")

    ds = xr.open_dataset(mapper, engine='zarr')

    datasets.append(ds)

ds = xr.concat(datasets, dim='slow_time', combine_attrs='drop_conflicts')
ds

### Add projected coordinates and compute stats we care about

In [ ]:
# Add x and y coordinates projected to EPSG:3031
projected_coords = epsg_3031.transform_points(
    crs.PlateCarree(), ds['Longitude'].values, ds['Latitude'].values
).T
ds = ds.assign_coords({
    'x': (('slow_time'), projected_coords[0]),
    'y': (('slow_time'), projected_coords[1])
})

In [ ]:
bed_minus_surface = ds.bed_power_dB - ds.surface_power_dB
bed_minus_surface = bed_minus_surface.dropna('slow_time')
bed_minus_surface.name = 'bed_minus_surface'

In [ ]:
h = ds.surface_twtt * scipy.constants.c / 2
h.name = 'surface_height'

n = np.sqrt(config['processing_flights']['ice_relative_permittivity'])
speed_in_ice = scipy.constants.c / n
z = (ds.bed_twtt - ds.surface_twtt) * speed_in_ice / 2
z.name = 'ice_thickness'

geom_spreading_surf_dB = 10 * np.log10(1 / (h**2))
geom_spreading_bed_dB = 10 * np.log10(1 / (h + z/n)**2)

rssnr = (
        (ds.surface_power_dB - geom_spreading_surf_dB) -
        (ds.bed_power_dB - geom_spreading_bed_dB)
        ).dropna('slow_time')
rssnr.name = 'rssnr'

In [ ]:
features = gf.ocean().options(scale='50m', projection=epsg_3031) * gf.coastline().options(scale='50m', projection=epsg_3031)

sc = rssnr.hvplot.scatter(x='x', y='y', c='rssnr', groupby=[], data_aspect=1, size=3,
                       hover_cols=['surface_power_dB', 'bed_power_dB'], cmap='turbo')

(features * sc).opts(
    width=800,
    height=600
)

In [ ]:
rssnr

In [ ]:
features = gf.ocean().options(scale='50m', projection=epsg_3031) * gf.coastline().options(scale='50m', projection=epsg_3031)

sc = bed_minus_surface.hvplot.scatter(x='x', y='y', c='bed_minus_surface', groupby=[], data_aspect=1, size=3,
                       hover_cols=['surface_power_dB', 'bed_power_dB'], cmap='turbo')

(features * sc).opts(
    width=800,
    height=600
)

In [ ]:
# features = gf.coastline().options(scale='50m') * gf.ocean().options(scale='50m') * gf.coastline().options(scale='50m')
# points = gv.Points(
#       data=(ds.Longitude, ds.Latitude, ds.surface_power_dB.values),
#       kdims=['Longitude', 'Latitude'],
#       vdims=['surface_power_dB'],
#       crs=latlon,  # Input CRS (lat/lon)
#   ).opts(
#       projection=epsg_3031,  # Output projection
#       color='surface_power_dB',
#       cmap='viridis',
#       colorbar=True,
#       hover_tooltips=['surface_power_dB'],
#       size=3,
#       width=800,
#       height=600,
#       data_aspect=1,
#       show_frame=True,
#       xaxis=True,
#       yaxis=True
#   )
# features * points